<font color=gray>Junio 2017 | Análisis de Redes Sociales con Python</font>

# <font color=#003d5c>Tweepy</font>

Para autorizar que nuestra aplicación acceda a twitter en nuestro nombre, necesitamos usar la interfaz de **OAuth**:

In [ ]:
import tweepy
import json
from tweepy import OAuthHandler

consumer_key = 'nkZF8L7QQsm3IqkjUgNRbzXSS'
consumer_secret = 'iDbgDcn8OvH3kZ1j7OuPBfFiCxv0v2CHY7GymGiCXCST0Ps7TB'
access_token = '2176488962-oyF8MhDTY9sw0KFVazXFnly4i6hQkupOtfX6Fk5'
access_secret = 'hcPfi0nAYGlVRWwAcwkpgAfKkRiqWjEGcAip4JRAs2y5q'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

La variable api es ahora nuestro punto de entrada para la mayorìa de las operaciones que podemos realizar con Twitter. Por ejemplo, podeemos leer nuestra propia línea de tiempo (es decir, nuestra **página principal de Twitter**) con:

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(2):
    print(status.text)

<p align="justify"> Además **Tweepy** proporciona una conveniente interfaz llamada **Cursor** para iterar a través de diferentes tipos de objetos. En el ejemplo anterior usamos 2 para limitar el número de tweets que estamos leyendo, pero podemos acceder a más. La variable **status** es una instancia de la clase Status(), una envoltura agradable para acceder a los datos.</p>

La respuesta JSON de la API de Twitter está disponible en el atributo **_json**, que no es una cadena, sino un diccionario. Así que el código anterior se puede volver a escribir para obtener el JSON:

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(1):
    print status._json

¿Qué pasa si queremos tener una lista de todos nuestros seguidores? Así lo hacemos:

In [ ]:
for friend in tweepy.Cursor(api.friends).items():
    print friend._json

¿Y qué tal una lista de todos nuestros tweets? Así de simple:

In [ ]:
for tweet in tweepy.Cursor(api.user_timeline).items():
    print tweet._json

<p align="justify"> De esta manera podemos recopilar fácilmente los tweets y almacenarlos en el formato original JSON, de tal manera que sea bastante fácil de convertir en diferentes modelos de datos dependiendo de nuestro almacenamiento (muchas tecnologías NoSQL proporcionan alguna característica de importación masiva).</p>

# <font color=#003d5c>Streaming</font>

<p align="justify"> En caso de que quisiéramos mantener la conexión abierta, y reunir todos los próximos tweets sobre un evento en particular, la API de streaming es lo que necesitamos. Necesitamos instanciar el **StreamListener()** para personalizar la forma en que procesamos los datos entrantes. Aquí se muestra un ejemplo de trabajo que reúne todos los nuevos tweets con el hashtag **#TransitoLima**:</p>

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('TransitoLima.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#TransitoLima'])

Dependiendo del término de búsqueda, podemos reunir toneladas de tweets en pocos minutos. Esto es especialmente cierto para eventos en vivo con una cobertura mundial, así que mantén un ojo en el archivo JSON para entender lo rápido que crece y considerar cuántos tweets puede necesitar para sus pruebas. 

El script anterior guardará cada tweet en una nueva línea, por lo que puede usar el comando **wc -l TransitoLima.json** de una shell de Unix para saber cuántos tweets ha reunido y **cat TransitoLima.json** para verlos.

<font color=gray>Referencia: </font> <a href="https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/#Register_Your_App">Mining Twitter Data with Python (Part 1: Collecting data)</a>